In [ ]:
regression_path = "regression_test_plan.md"
with open(regression_path, "r") as f:
    regression_template_lines = f.readlines()


In [ ]:
import re
from typing import List

def parse_metadata(lines:List[str]):
    metadata = None

    for line_num, line in enumerate(lines):
        if line.strip() == "---":
            if metadata == None:
                metadata = dict()
            else:
                return metadata, line_num
        else:
            metadata_match = re.match(r"(.+):(.+)", line)
            if metadata_match:
                key = metadata_match.group(1).strip()
                text = metadata_match.group(2).strip()
                if key in metadata:
                    raise ValueError("duplicate metadata key")
                metadata[key] = text

print(parse_metadata(regression_template_lines))
 

In [ ]:
md,ln=parse_metadata(regression_template_lines)
print(md)
print(ln)
print(regression_template_lines[ln+1:])

def is_new_line(line):
    return line.strip() == ""

print(is_new_line(regression_template_lines[ln]))
print(is_new_line(regression_template_lines[ln+1]))
print(is_new_line(regression_template_lines[ln+2]))

In [ ]:
import json
import re

def is_table(line):
    return re.match(r"\|.*\|", line)

def is_heading(line):
    return re.match(r"^(#+) (.+)", line)

# list_match = re.match(r"^[-*+] (.+)", line)

def parse_regression_template(lines):
    metadata, line_num = parse_metadata(lines)
    if metadata == None:
        raise ValueError("metadata not found")
    
    result = {
        "metadata": metadata,
        "content": {}
    }
    
    heading_dict = None
    for line_num, line in enumerate(lines[line_num+1:]):

        heading_match = is_heading(line)
        # print(heading_match)
        if heading_match:
            hlevel = len(heading_match.group(1))  # Number of `#`
            htitle = heading_match.group(2).strip()
            heading_dict = result["content"][line_num] = {"headingLevel": hlevel, "headingTitle": htitle, "content": []}

        if not heading_dict:
            if len(line.strip()) == 0:
                continue
            heading_dict = result["content"][line_num] = {"headingLevel": 0, "headingTitle": "", "content": []}

        heading_dict["content"].append(line.strip())
    
    return result
  
 
parsed = parse_regression_template(regression_template_lines)

print(json.dumps(parsed, indent=2))


In [ ]:
# in parsed template, we have metadata and content
# content dictionary has key as line number.
# there is content array in value which is useful to format template in markdown
# we can use this to generate test plan in markdown format
# content array has variables usage in the template denoting $variable_name
# only table of test scenario is special. we will need to add rows to the table

In [ ]:
import json
# step1: call convert tc and summary methods to get list of test cases
# python .\.github\scripts\parse_test_cases.py --convert --tc-dir test-cases\ --converted-filename playground3-converted-tcs
with open("../../../dist/playground3-converted-tcs.json", "r") as f:
    converted_tcs = json.load(f)

# step2: generate summary for type of test cases
# python .\.github\scripts\summary_tc.py --analyze --converted-tc-path dist\playground3-converted-tcs.json --summary-filename playground3-summary-typeoftest --key-path "details.Type of Test"

with open("../../../dist/playground3-summary-typeoftest.json", "r") as f:
    summary_typeoftest = json.load(f)

regression_testcases = summary_typeoftest["Regression"]




In [ ]:
from collections import defaultdict, Counter
from typing import List, Dict

def group_list_into_map(items: List[str]) -> Dict[str, List[str]]:
    keyword_count = Counter()
    keyword_map = defaultdict(set)

    exclude_keywords = [ "api", "form", "add", "page" ]
    # Count occurrences of each keyword
    for item in items:
        keywords = item.split()
        for keyword in keywords:
            if keyword in exclude_keywords:
                continue
            keyword_count[keyword] += 1
            keyword_map[keyword].add(item)

    # Sort keywords by their occurrences in descending order
    sorted_keywords = sorted(keyword_count.keys(), key=lambda k: -keyword_count[k])

    # Create the final map with unique values
    final_map = {}
    seen_items = set()

    for keyword in sorted_keywords:
        unique_items = [item for item in keyword_map[keyword] if item not in seen_items]
        if unique_items:
            final_map[keyword] = unique_items
            seen_items.update(unique_items)

    return final_map


In [ ]:
import re

def get_variable_name(content):
    return re.findall(r"\$[a-zA-Z0-9_\.]+", content)

get_variable_name("This is $variable1 and $var.able.2")

def is_list(content):
    return re.match(r"^[-*+] (.+)", content)


def get_value(variable:str, tc_id:str=None):
    testcases = regression_testcases if tc_id is None else [tc_id]
    ia_list = set()
    for tc in testcases:
        if variable == "$details.impact_area":
            for k,v in converted_tcs[tc]["details"]["Impact Area"].items():
                ia_list.add(k)
                for lv in v:
                    ia_list.add(lv)
        if variable == "$details.tags.feature":
            ia_list.update([v for v in  converted_tcs[tc]["details"]["Tags"]["feature"]])
        if variable == "$details.tags.impact":
            ia_list.update([fv for fv in converted_tcs[tc]["details"]["Tags"]["impact"]])
        if variable == "$details.title":
            ia_list.add(converted_tcs[tc]["details"]["Title"])
        if variable == "$metadata.id":
            ia_list.add(converted_tcs[tc]["metadata"]["id"])
        if variable == "$metadata.relative_file_path":
            ia_list.add(converted_tcs[tc]["metadata"]["relative_file_path"].replace("\\", "/"))
    if len(ia_list) == 0:
        return [variable]
    return sorted(ia_list)

def replace_variables(content_line:str, variables:List[str], tc_id:str=None):
    content = ""
    if is_list(content_line) and len(variables) == 1:
        var = variables[0]
        ll = get_value(var, tc_id)
        gmap = group_list_into_map(ll)
        clist = []
        for gv in gmap.values():
            clist.append(content_line.replace(var, ", ".join(gv)))
        content = "\n".join(clist)
    else:
        content = content_line
        for var in variables:
            ll = get_value(var, tc_id)
            content = content.replace(var, ", ".join(ll))
    return content

In [ ]:
def get_milestone_details(milestone:str):
    
    return {
        "milestone": milestone,
        "milestone_start": "2021-06-01",
        "milestone_end": "2021-06-30",
        "milestone_duration": 30
    }

In [ ]:
sorted_line_numbers = sorted(parsed["content"].keys())

file_contents = []

for line_num in sorted_line_numbers:
    content = parsed["content"][line_num]["content"]
    variables_in_content = list()
    
    for content_line in content:
        variables = get_variable_name(content_line)
        variables_in_content.extend(variables)
        if "$ind" in variables and parsed["content"][line_num]["headingTitle"].startswith("Test Scenarios for Regression"):
            for ind, tc in enumerate(regression_testcases):
                tc_content = content_line.replace("$ind", str(ind+1))
                file_contents.append(replace_variables(tc_content, variables, tc))
        else:
            file_contents.append(replace_variables(content_line, variables))

    # print(variables_in_content)
        # for variable in variables:
        #     if variable not in parsed["metadata"]:
        #         print(f"Variable {variable} not found in metadata")
        #     else:
        #         content_line = content_line.replace(variable, parsed["metadata"][variable])
        # parsed["content"][line_num]["content"] = content_line
# print(file_contents)

# print("\n".join(file_contents))

with open("../../../dist/regression_test_plan_generated.md", "w") as f:
    f.write("\n".join(file_contents))


In [ ]:
import re
testplan_content = "Regression Test Plan: [testplan issue-12](https://github.com/rajexcited/personal-finance-ui/issues/12)"
linkMatch = re.match(r"([a-zA-Z]+) Test Plan:.+https.+/issues/(\d+).*",
                         testplan_content, re.IGNORECASE)

if linkMatch:
    print(linkMatch.group(1))
    print(linkMatch.group(2))
else:
    print("No match")

In [ ]:
from pathlib import Path
import json

request_form_path=Path("../../../../finance-journal-ui/dist/request_form_issue_details.json")
print(request_form_path.resolve(), request_form_path.exists())

with open(request_form_path, "r") as rf:
    request_form_json = json.load(rf)

print(request_form_json["body"])


In [ ]:
from markdown_to_json import dictify
body:str=request_form_json["body"]
lines=body.split("\n")
lines.insert(1,"## Dummy")
print("\n".join(lines))
parsed=dictify("\n".join(lines))
ignorek, request_dict_parsed=parsed.popitem()
# type(request_dict_parsed)
print(type(request_dict_parsed))
print(len(request_dict_parsed))
for k,v in request_dict_parsed.items():
    print(k, type(v))
    for kk,vv in v.items():
        print(kk, type(vv), vv)

In [ ]:
print(ignorek)
# print(body)
ind=body.find(ignorek)
# print(ind+len(ignorek))
header1_end_ind=body.find("\n", ind+len(ignorek))
print("header1 ends at: ", header1_end_ind)

print(body[:header1_end_ind] + "\n## Dummy" + body[header1_end_ind:])

In [ ]:
from typing import List,Dict

def parsed_body(requestform_body: str, header1_dummy: bool = False, header2_dummy: bool = False) -> List:
    """
    Args:
        requestform_body (str):  request form issue body
        header1_dummy and header2_dummy are used for internal to correct the body to parse in dictionary
    Returns:
        list of header3 types, parsed header3 could be of any of these types; list, dict or str.
    """
    parsed_req_form = dictify(requestform_body)
    if not isinstance(parsed_req_form, Dict) and not header1_dummy:
        # add header1 and retry
        return parsed_body("# Dummy\n"+requestform_body, header1_dummy=True)

    incorrect_format_error_message = "request form is not in correct format. Please follow template `Request  Regression - Provision/Deprovision Test Plan Environment`"

    if not isinstance(parsed_req_form, Dict) or len(parsed_req_form) != 1:
        raise TypeError(incorrect_format_error_message)

    ignorek, header1_value = parsed_req_form.popitem()
    if not isinstance(header1_value, Dict) and not header2_dummy:
        # add header2 and retry
        header1_index = requestform_body.find(ignorek)
        header1_end_index = requestform_body.find(
            "\n", header1_index+len(ignorek))
        return parsed_body(requestform_body[:header1_end_index] + "\n## Dummy" + requestform_body[header1_end_index:], header2_dummy=True)

    if not isinstance(header1_value, Dict) or len(header1_value) != 1:
        raise TypeError(incorrect_format_error_message)

    ignorek, header2_value = header1_value.popitem()
    if not isinstance(header2_value, Dict):
        print(type(header2_value), header2_value)
        raise TypeError(incorrect_format_error_message)

    return header2_value


In [ ]:
print(parsed_body(body))
